In [12]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import os
from nltk.tokenize import sent_tokenize
import pandas as pd

In [13]:
import config
os.getcwd()

'/Users/mark/src/books/src'

In [14]:
os.listdir(config.dataset_dir)

['.DS_Store', 'output', 'test']

Sentiment analysis. Analysis is performed for each sentence and the sentiment scores kept in lists. Sentiment scores are calculated by averaging the sentiment scores for all sentences.

In [17]:
def return_sentiment_scores(sentence):
    # return just the sentiment scores
    snt = analyser.polarity_scores(sentence)
    return snt

def sentiment_analysis(directory):
    # returns the sentiment of every book in the directory
    data = pd.read_csv(config.dataset_dir + 'output/tard.csv', index_col=0)
    print(len(data.index))
    max_amt = len(data.index) + 1
    print(data.index, len(os.listdir(directory)))
    pos_list = []
    neg_list = []
    neu_list = []
    comp_list = []
    
    # for every book
    for filename in os.listdir(directory)[:max_amt]:
        
        sub_pos_list = []
        sub_neg_list = []
        sub_neu_list = []
        sub_comp_list = []
        
        # if file is a textfile
        if filename.endswith(".txt"):
            text = open(os.path.join(directory, filename), 'r', errors='replace')
            # for every line in the text
            for line in text.readlines():
                scores = return_sentiment_scores(line)
                # save sentiment scores 
                sub_neg_list.append(scores['neg'])
                sub_neu_list.append(scores['neu'])
                sub_pos_list.append(scores['pos'])
                sub_comp_list.append(scores['compound'])
            
            # then save average sentiment scores for each book
            neg_list.append((sum(sub_neg_list) / float(len(sub_neg_list))))
            pos_list.append((sum(sub_pos_list) / float(len(sub_pos_list))))
            neu_list.append((sum(sub_neu_list) / float(len(sub_neu_list))))
            comp_list.append((sum(sub_comp_list) / float(len(sub_comp_list))))
            
    # convert scores to pandas compatible list
    neg = pd.Series(neg_list)
    pos = pd.Series(pos_list)
    neu = pd.Series(neu_list)
    com = pd.Series(comp_list)

    print(len(neg), len(pos), len(neu), len(com))
    # fill the right columns with the right data
    print(type(data),'type')
    print(neg)
    data['neg score'] = neg.values
    data['pos score'] = pos.values
    data['neu score'] = neu.values
    data['comp score'] = com.values
    return data

#     data.to_csv(config.dataset_dir + 'output/tard.csv')

In [18]:
analyser = SentimentIntensityAnalyzer()                    
sentiment_analysis(config.dataset_dir + 'output/')

18
Index(['00053.txt', '00054.txt', '00055.txt', '00056.txt', '00057.txt',
       '00058.txt', '00059.txt', '00060.txt', '556.txt', '557.txt', '558.txt',
       '559.txt', '560.txt', '561.txt', '562.txt', '563.txt', '564.txt',
       '570.txt'],
      dtype='object', name='filename') 21
17 17 17 17
<class 'pandas.core.frame.DataFrame'> type
0     0.000000
1     0.016808
2     0.041491
3     0.052800
4     0.014526
5     0.058486
6     0.073558
7     0.037773
8     0.058397
9     0.042928
10    0.041528
11    0.046384
12    0.054632
13    0.054664
14    0.063559
15    0.031213
16    0.028252
dtype: float64


ValueError: Length of values does not match length of index